The full functionality of `seaflowpy` can be found through the various submodules, e.g. `seaflowpy.db`, `seaflowpy.evt`. However, for convenience a few of the most commonly-used functions and classes are exposed at the package level.

* **seaflowpy.EVT**  
Class for EVT particle data

* **seaflowpy.find_evt_files**  
Function to recursively find EVT/OPP file paths within a directory

The code below will use the test dataset in this repository at `./tests/testcruise/`. The files in this directory hold raw EVT data, but in this example workflow we'll treat the files as though they actually contain filtered OPP data. For example, we may have already filtered EVT to OPP data on the command-line with `seaflowpy_filter`.

A note on terminology: In this package the phrase EVT can have two subtley different meanings.  

* Any binary file or Python data structure which holds SeaFlow particle data, regardless of whether raw or filtered.
* In the context of a filtering workflow, EVT refers to the raw, unfiltered/unfocused version of particle data, distinct from OPP data which refers to the filtered/focused particles.  

When filtering, we talk about converting EVT data to OPP data. We may read a raw EVT file into Python as a `seaflowpy.EVT` object and the raw particle data is stored as a pandas DataFrame in the `EVT.df` attribute. We then filter the raw particle data with `seaflowpy.EVT.filter` and this filtered particle data is returned as a new EVT object representing OPP data. `seaflowpy_filter` is a convenient command-line wrapper for this process that can operate on whole cruises at a time.

In [1]:
import seaflowpy as sfp

In [2]:
opp_files = sfp.find_evt_files("./tests/testcruise_opp")
opp_files

['./tests/testcruise_opp/2014_185/2014-07-04T00-00-02+00-00.opp.gz',
 './tests/testcruise_opp/2014_185/2014-07-04T00-03-02+00-00.opp.gz']

Let's read the EVT files into memory. In many cases we don't plan on using all 10 channels of particle data, so here we'll select only three of the possible ten channels (columns). This can significantly speed up data import when transforming (exponentiating log data) and lowers the memory footprint.

In [3]:
# The possible column names to choose from
sfp.EVT.all_columns

['time',
 'pulse_width',
 'D1',
 'D2',
 'fsc_small',
 'fsc_perp',
 'fsc_big',
 'pe',
 'chl_small',
 'chl_big']

In [4]:
opps = []
for f in opp_files:
    opps.append(sfp.EVT(f, transform=True, columns=["fsc_small", "chl_small", "pe"]))

Now we have some EVT objects in `opps`. We can print an one of the `EVT` objects in `opps` to get a quick summary of it's content.

In [5]:
print opps[0]

{
  "path": "./tests/testcruise_opp/2014_185/2014-07-04T00-00-02+00-00.opp.gz", 
  "file_id": "2014_185/2014-07-04T00-00-02+00-00", 
  "header_count": 386, 
  "event_count": 386, 
  "particle_count": 386, 
  "columns": [
    "fsc_small", 
    "pe", 
    "chl_small"
  ], 
  "filter_options": {
    "origin": null, 
    "width": null, 
    "notch1": null, 
    "notch2": null, 
    "offset": null
  }
}


The underlying particle data can be accessed as a pandas DataFrame in the `df` attribute.

In [6]:
opps[0].df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 3 columns):
fsc_small    386 non-null float64
pe           386 non-null float64
chl_small    386 non-null float64
dtypes: float64(3)
memory usage: 9.1 KB


In [7]:
opps[0].df.head()

,fsc_small,pe,chl_small
0,5.910258,1.207901,7.622597
1,1.569201,1.399968,3.347019
2,73.487071,1.660321,74.359745
3,2.330406,1.275528,5.276099
4,2.386083,1.172052,7.774060


Let's assume this data set has already been analyzed and population classifications exist in a directory called `./tests/testcruise_vct`. We can add these per-particle classifications to our `EVT` objects with `seaflowpy.EVT.add_vct`.

`add_vct` can take a directory as its only argument. In order to find the appropriate VCT file the structure of this directory should mirror the OPP and raw EVT directories (julian day/file).

`add_vct` can also take a path to a VCT file if the file has an unconventional name or location.

In [11]:
for opp in opps:
    opp.add_vct("./tests/testcruise_vct")
opps[0].df.head()  # a new "pop" column has been added

,fsc_small,pe,chl_small,pop
0,5.910258,1.207901,7.622597,prochloro
1,1.569201,1.399968,3.347019,prochloro
2,73.487071,1.660321,74.359745,picoeuk
3,2.330406,1.275528,5.276099,prochloro
4,2.386083,1.172052,7.774060,prochloro


Now we can operate on the pandas particle DataFrame directly to get per-population statistics ...

In [12]:
bypop = opps[0].df.groupby("pop")
bypop.describe()

chl_small    fsc_small           pe
pop                                                   
beads     count    62.000000    62.000000    62.000000
          mean     61.331928    65.510412   703.436398
          std      18.317616    16.732172   186.882179
          min      25.481625    28.523328   340.508047
          25%      51.075239    58.808637   620.671852
          50%      60.147136    62.292189   685.371556
          75%      66.955352    66.079745   738.692231
          max     135.056675   131.306472  1318.311393
picoeuk   count    20.000000    20.000000    20.000000
          mean    290.085973   163.255617     1.836653
          std     330.518240    96.619975     1.069171
          min      53.818816    32.928968     1.163579
          25%      95.702179    77.999994     1.294704
          50%     215.288536   172.353941     1.544872
          75%     349.497415   206.037612     1.714948
          max    1540.007685   357.456054     5.802952
prochloro count   221.000000   221.000000   221.000000
          mean      4.240353     3.581225     1.435859
          std       2.822278     3.436110     3.163474
          min       1.407390     1.161292     1.000000
          25%       2.175326     2.141354     1.088286
          50%       3.454062     2.738420     1.203157
          75%       5.367058     3.664079     1.310185
          max      23.077144    29.726552    48.168392
synecho   count    78.000000    78.000000    78.000000
          mean     10.070132    17.089432     9.630957
          std       8.209542    22.624491     6.889422
          min       1.380819     2.727665     2.201159
          25%       4.940535     8.543895     4.315614
          50%       8.287975    11.504535     7.731612
          75%      12.550341    16.653181    12.094722
          max      50.237372   173.288416    36.212689
unknown   count     5.000000     5.000000     5.000000
          mean      5.433694   356.833772     4.404959
          std       4.887757   464.046203     3.279403
          min       1.334576    51.267114     1.486004
          25%       2.172386    93.057204     2.462393
          50%       2.345644   151.587735     2.729343
          75%       9.243644   322.058355     5.863926
          max      12.072219  1166.198453     9.483128

In [13]:
bypop.mean()

,fsc_small,pe,chl_small
pop,,,
beads,65.510412,703.436398,61.331928
picoeuk,163.255617,1.836653,290.085973
prochloro,3.581225,1.435859,4.240353
synecho,17.089432,9.630957,10.070132
unknown,356.833772,4.404959,5.433694


In [14]:
bypop.size()

pop
beads         62
picoeuk       20
prochloro    221
synecho       78
unknown        5
dtype: int64

Or we can call the `calc_pop_stats` method of the EVT object to get the same summary statistics that are saved in the SQLite3 database in a full classification workflow. This returns a dictionary keyed by population name with particle counts and per-channel means for fsc_small, fsc_perp, pe, chl_small. If one of these columns is not present in the EVT object, as fsc_perp is not here, it's left out of the results.

In [15]:
import pprint
pprint.pprint(opps[0].calc_pop_stats())

{'beads': {'chl_small': 61.331927505955953,
           'count': 62,
           'fsc_small': 65.510412124807516,
           'pe': 703.43639797379763,
           'pop': 'beads'},
 'picoeuk': {'chl_small': 290.08597340883045,
             'count': 20,
             'fsc_small': 163.2556167296695,
             'pe': 1.836653035695474,
             'pop': 'picoeuk'},
 'prochloro': {'chl_small': 4.240352677583366,
               'count': 221,
               'fsc_small': 3.5812246921776758,
               'pe': 1.4358590192656131,
               'pop': 'prochloro'},
 'synecho': {'chl_small': 10.070132102414576,
             'count': 78,
             'fsc_small': 17.089432037286574,
             'pe': 9.6309571133821361,
             'pop': 'synecho'},
 'unknown': {'chl_small': 5.4336937909910636,
             'count': 5,
             'fsc_small': 356.83377202864756,
             'pe': 4.404958684938495,
             'pop': 'unknown'}}
